Notebook to validate if recovered secret is correct. 
You can also just use the built-in checks in evalutor.py
See Section 4.4 in our paper for more details.

In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
# 
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import circulant

def rlwe_sample(s, Q, N, rng, sigma=3, a=None):
    if a is None:
        a = rng.integers(0, Q, size=N, dtype=np.int64)
    # do the circulant:
    c = circulant(a)
    tri = np.triu_indices(N, 1)
    c[tri] *= -1
    c = c % Q # EJWQ
    e = np.int64(rng.normal(0, sigma, size = N).round())
    b = (np.inner(c, s) + e) % Q
    return a, b

def gen_secret(N, hamming, rng):
    s = np.zeros(shape=N, dtype=np.int64)
    for _ in range(hamming):
        setit = False
        while not setit:
            idx = rng.integers(N, size=1)
            if s[idx] != 1:
                s[idx] = 1
                setit = True
        return s

If s is correct secret and t is an incorrect guess:  
Var(b-as) = Var(e)=sigma^2,   
Var(b-at) = Var(Uniform(O,Q))= Q^2/12  
with q=251 and sigma=3, Var(b-as)=9 and Var(b-at)=5250  

In [2]:
Q = 251
N=50
hamming=5
sigma=np.sqrt(N)
rng = np.random.default_rng()
s = gen_secret(50, 5, rng)

In [3]:
origA = []
origB = []
for i in range(10):
    a, b = rlwe_sample(s, Q, N, rng, sigma, None)
    origA.append(a)
    origB.append(b)

In [4]:
guess_s = s # gen_secret(50, 5, rng)#s
guess_s[3] = 1
def modFixer(x):
    if x > Q-(sigma**2):
        return Q-x
    return x

for i in range(5):
    newB = []
    for a in origA:
        _, b = rlwe_sample(guess_s, Q, N, rng, sigma, a)
        newB.append(b)
    diffBs = (np.array(origB) - np.array(newB)) % Q
    diffBs_fixed = np.vectorize(modFixer)(diffBs)
    print('test', i, 'std', np.std(diffBs_fixed))

test 0 std 60.62152551693168
test 1 std 60.153520229492806
test 2 std 61.41775831142
test 3 std 60.51078370009762
test 4 std 61.34926434766761


In [59]:
np.std(diffBs_fixed)

21.246027769915013